## Preprocessing features

Preprocessing aim is to generate a 2D data structure (a table) that can go into catboost training.

### Sampling point locations

The WorldCereal extractions cache consists of 64x64 pixel timeseries stored as netCDF files.
As catboost is a 1D method, we need to sample those patches at point locations.

In the approach presented in the code block below, the original algorithm is translated into an openEO process graph.
It is however also possible to come up with other approaches, for instance that sample the patches at point locations, and then perform a stratification step on the larger dataset.



In [3]:
import  openeo
connection = openeo.connect("openeo.dataspace.copernicus.eu")
ground_truth = connection.load_stac("https://stac_catalog.com/ground_truth")

from   openeo import UDF
sampling_udf=UDF(code="",runtime="Python") # <1>

polygons = {"type":"FeatureCollection"} #these would be the bounding boxes of the netCDF files, or in fact STAC item bboxes

ground_truth.apply_polygon(polygons,process=sampling_udf)

1. This UDF should return points as geojson


### Extracting point timeseries

In [4]:
import  openeo
from    openeo.rest.mlmodel import MlModel
from    openeo.processes import ProcessBuilder

connection = openeo.connect("openeo.dataspace.copernicus.eu")
l2A = connection.load_stac("https://stac_catalog.com/SENTINEL2_L2A").aggregate_temporal_period(period="month",reducer="mean")  # <1>
sentinel1 = connection.load_stac("https://stac_catalog.com/SENTINEL1_BS")
bs = sentinel1.aggregate_temporal_period(period="month",reducer="mean")

timesteps_cube = l2A.merge_cubes(bs).aggregate_spatial(geometries={"type":"Point"},reducer="mean").save_result(format="Parquet")
# <2>

timesteps_cube


1. instead of aggregate_temporal, we'll do more advanced compositing, such as max-NDVI
2. we'll need to add agera5 and dem bands

### Training workflow

The training workflow combines feature computation starting from monthly timesteps with catboost training.

In [6]:
from    openeo import UDF
feature_udf=UDF(code="",runtime="Python") #load UDF to compute presto features based on monthly timeseries
features_cube = connection.load_url("timesteps.parquet",format="Parquet").apply_dimension(dimension='t',process=feature_udf,target_dimension='bands')
ml_model = features_cube.process("fit_catboost_model")


## Extracting private samples




In [ ]:
sample_locations = connection.load_url("https://rdm.worldcereal.org/private_assets/absqdfjq_signed_url/private_data.parquet")

connection.datacube_from_process("worldcereal_preprocessing_udp").filter_spatial(sample_locations).save_result(format="Parquet")



## Training by combining private + public samples

In this usecase, the user wants to train a new model, by combining data.
This should be possible by simply merging vector cubes that go into the training process.

TODO